In [ ]:
import pandas as pd
import multiprocessing as mp
import numpy as np
from datetime import datetime

In [ ]:
test = ["D:\homework2\hw2-rdf-2016_test\hw2-rdf-2016_test.txt"]
files = []
for i in range(1,2):
    files.append("D:\homework2\data\hw2-rdf-2016_1\hw2-rdf-2016_{}.dat".format(str(i)))

Function that does the work

In [ ]:
def dothejob(input,finaldf):
    #tdf = pd.DataFrame(columns = ['SUBJECT', 'EDGES'])
    for f in input:
        with open(f,'r') as f:
            i = 0
            reader = pd.read_table(f, header=None, chunksize = chunkk)
            for df in reader:
                #print(df)
                df = df[0].str.split(" ", expand=True)
                df['EDGES'] = df.stack().str.contains(r'^\"|<|_:').astype(np.uint8).sum(level=0)-1
                df[df['EDGES'] >= 3] = 3
                df[df['EDGES'] <= 2] = 2
                df.rename(columns={0:'SUBJECT'}, inplace=True)
                finaldf = finaldf.append(df[['SUBJECT','EDGES']], ignore_index=True)
                # Delete the original file dataframe
                del df
                #finaldf = finaldf.groupby('SUBJECT', as_index=False).sum()
                i += chunkk
                print("\nCompleted {} rows frol file {}".format(str(i),str(f).split("'")[1]))   
                #finaldf = finaldf.append(df[['SUBJECT','EDGES']], ignore_index=True)        
                finaldf = finaldf.groupby('SUBJECT', as_index=False).sum()
    return finaldf

In [ ]:
tdf = pd.DataFrame(columns = ['SUBJECT', 'EDGES'])
chunkk = 50000
t1=datetime.now()
tdf = dothejob(files,tdf)    # WILL RUN FOR FIRST FILE ONLY
print("\n Execution time: {}".format(datetime.now() - t1))

#tdf['y'] = 1
#tdf = tdf[['EDGES','y']].groupby('EDGES', as_index=False).sum()
print(tdf)
print(tdf.shape)